In [68]:
# Import SparkConf, RDD, SparkContext, SparkSession class into program
from pyspark import SparkConf
from pyspark.rdd import RDD
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import numpy as np
import csv

# Using all the processor 
master = "local[*]"
# Initiallizing app name
app_name = "Task 1"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# instantiating a SparkContext
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

**Import all the “Units” csv files from 2015-2019 into a single RDD.**

**Import all the “Crashes” csv files from 2015-2019 into a single RDD**.

In [69]:
# loading all the Unit and Crash data into single RDD respectively
Unit_RDD = sc.textFile('data/*_Units.csv')
Crash_RDD = sc.textFile('data/*_Crash.csv')

**For both Units and Crashes, use csv.reader to parse each row as a nested list in the
RDD, and remove the header rows and display the total count and first 10 records.**

In [70]:
# selecting header row from the RDD
UnitHeader = Unit_RDD.first()
CrashHeader = Crash_RDD.first()
# removing the header row for both Unit and Crash RDD object
Unit_RDD = Unit_RDD.filter(lambda x: x != UnitHeader)
Crash_RDD = Crash_RDD.filter(lambda x: x != CrashHeader)

In [71]:
# creating parser to prse the data
def parser(line):
    lines = line.splitlines()
    read = csv.reader(lines, delimiter=',')
    for row in read:
        x = ','.join(row)
    return x.split(',')

# Parsing unit and Crash data
Unit_RDD1 = Unit_RDD.map(parser)
Crash_RDD1 = Crash_RDD.map(parser)

print("Total Count for Unit Data: ",Unit_RDD1.count())
print("Total Count for Crash Data: ",Crash_RDD1.count(), "\n")

print("First 10 records for Unit Data \n",Unit_RDD1.take(10))
print("\nFirst 10 records for Crash Data \n", Crash_RDD1.take(10))

Total Count for Unit Data:  153854
Total Count for Crash Data:  72006 

First 10 records for Unit Data 
 [['2016-1-15/08/2019', '01', '0', 'SA', 'OMNIBUS', '2011', 'North', 'Male', '056', 'SA', 'HR', 'Full', 'Not Towing', 'Straight Ahead', '010', '5121', '', ''], ['2016-1-15/08/2019', '02', '1', '', 'Pedestrian on Road', '', 'East', 'Male', '072', '', '', '', '', 'Walking on Road', '', '5084', '', ''], ['2016-2-15/08/2019', '01', '0', 'SA', 'Motor Cars - Sedan', '2004', 'Unknown', 'Female', '023', 'SA', 'C ', 'Full', 'Not Towing', 'Straight Ahead', '001', '5087', '', ''], ['2016-2-15/08/2019', '02', '0', 'SA', 'Station Wagon', '2008', 'Unknown', 'Male', '040', 'SA', 'C ', 'Full', 'Not Towing', 'Straight Ahead', '001', '5084', '', ''], ['2016-3-15/08/2019', '01', '0', 'SA', 'RIGID TRUCK LGE GE 4.5T', '1990', 'South', 'Unknown', 'XXX', 'SA', 'MR', 'Provisional 2', 'Not Towing', 'Straight Ahead', '001', '5115', '', ''], ['2016-3-15/08/2019', '02', '0', 'SA', 'Panel Van', '2013', 'South', 

**1.2 Data Partitioning in RDD**

**How many partitions do the above RDDs have? How is the data in these RDDs
partitioned by default, when we do not explicitly specify any partitioning strategy?**

**Ans: By default there are 5 partitions. Data in these partitions are almost equally distributed but with some partitions having more data than others**

In [72]:
# Printing default partitions in data
print('Default partitions for Unit: ',Unit_RDD1.getNumPartitions())
print('Default partitions for Crash: ',Crash_RDD1.getNumPartitions())

# collecting partitions for Unit data
Unit_partition = Unit_RDD1.glom().collect()

print("\nPartition for Unit data")
for index, part in enumerate(Unit_partition):
    if len(part) > 0:
        print("Partition", index, "No of records:", len(part))

# collecting partitions for Crash data
Crash_partition = Crash_RDD1.glom().collect()

print("\nPartition for Crash data")
for index, part in enumerate(Crash_partition):
    if len(part) > 0:
        print("Partition", index, "No of records:", len(part))

Default partitions for Unit:  5
Default partitions for Crash:  5

Partition for Unit data
Partition 0 No of records: 35861
Partition 1 No of records: 28163
Partition 2 No of records: 33084
Partition 3 No of records: 27713
Partition 4 No of records: 29033

Partition for Crash data
Partition 0 No of records: 12964
Partition 1 No of records: 16775
Partition 2 No of records: 13237
Partition 3 No of records: 13599
Partition 4 No of records: 15431


**Create a Key Value Pair RDD with Lic State as the key and rest of the column as
value.**

In [73]:
# Function for Key value pair
def keyValue(li):
    x = li.pop(9)
    return (x, li[:])

# Creating key value pair RDD with Lic State as key
Unit_RDD_keyValue = Unit_RDD1.map(keyValue)

**Write the code to implement this partitioning in RDD using appropriate
partitioning functions.**

**Write the code to print the number of records in each partition. What does it
tell about the data skewness?**

**Ans: Lic State equals to 'SA' has 109683 records and other states have 44175 records. We can clearly see the data skweness as most of the records belongs to 'SA' state. Since we have done partitining based on state we have data skweness**

In [74]:
NoOfPartition = 2

# partitining function based on Lic State equals to 'SA' 
def hash_partition(key):
    if key == 'SA':
        return 0
    else:
        return 1

# Creating two partitions based on Lic State
Unit_RDD2 = Unit_RDD_keyValue.partitionBy(NoOfPartition,hash_partition)

partition = Unit_RDD2.glom().collect()

for index, part in enumerate(partition):
    if len(part) > 0:
        print("Partition", index, "No of records:", len(part))


Partition 0 No of records: 109684
Partition 1 No of records: 44170


**1.3 Query/Analysis ( 10% )**

**Find the average age of male and female drivers separately.**

In [75]:
# Filtering RDD based on Male and Female
Age_Male = Unit_RDD2.filter(lambda x: (x[1][7]=='Male') and (x[1][8]!='XXX'))
Age_Female = Unit_RDD2.filter(lambda x: x[1][7]=='Female' and (x[1][8]!='XXX'))

# Function to calculate Mean age
def MeanAge(data):
    age = data.collect()
    i = 0
    total = 0
    for value in age:
        total += int(value[1][8])
        i+=1
    meanAge = total/i
    return meanAge

print("Mean age for Males: ", round(MeanAge(Age_Male),2))
print("\nMean age for Females", round(MeanAge(Age_Female),2))

Mean age for Males:  40.98

Mean age for Females 40.39


**What is the oldest and the newest vehicle year involved in the accident? Display the
Registration State, Year and Unit type of the vehicle.**

In [76]:
# Removing unwanted data rows
cleanData = Unit_RDD2.filter(lambda x: (x[1][5]!='XXXX') and x[1][5]!='')
# finding max year
result_max_year = cleanData.max(key=lambda x: int(x[1][5]))
# finding min year
result_min_year = cleanData.min(key=lambda x: int(x[1][5]))
# filtering Registration State, Year and Unit type
result_max_year = result_max_year[1][3:6]
result_min_year = result_min_year[1][3:6]

print("Newest Vehicle:")
print(result_max_year)
print("\nOldest Vehicle:")
print(result_min_year)

Newest Vehicle:
['SA', 'Station Wagon', '2019']

Oldest Vehicle:
['SA', 'Motor Cycle', '1900']


**2. Working with DataFrames**

In [77]:
# reading all Unit data in dataframe
df_Units = spark.read.format('csv')\
            .option('header',True).option('escape','"')\
            .load('data/*Units.csv')

In [78]:
# reading all Crash data in dataframe
df_Crash = spark.read.format('csv')\
            .option('header',True).option('escape','"')\
            .load('data/*Crash.csv')

**Display the schema of the final two dataframes.**

In [79]:
# printing Schema for both Unit and Crash Data
print("Unit Schema")
df_Units.printSchema()
print("\nCrash Schema")
df_Crash.printSchema()

Unit Schema
root
 |-- REPORT_ID: string (nullable = true)
 |-- Unit No: string (nullable = true)
 |-- No Of Cas: string (nullable = true)
 |-- Veh Reg State: string (nullable = true)
 |-- Unit Type: string (nullable = true)
 |-- Veh Year: string (nullable = true)
 |-- Direction Of Travel: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Lic State: string (nullable = true)
 |-- Licence Class: string (nullable = true)
 |-- Licence Type: string (nullable = true)
 |-- Towing: string (nullable = true)
 |-- Unit Movement: string (nullable = true)
 |-- Number Occupants: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Rollover: string (nullable = true)
 |-- Fire: string (nullable = true)


Crash Schema
root
 |-- REPORT_ID: string (nullable = true)
 |-- Stats Area: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- LGA Name: string (nullable = true)
 |-- Total Units:

**Find all the crash events in Adelaide where the total number of casualties in the event
is more than 3.**

In [80]:
# converting Total cas data to int
df_Crash['Total Cas'].cast(IntegerType())
# filtering data based on ADELAIDE suburd and total cas greater than 3
df_Adelaide = df_Crash[(df_Crash['Suburb'] == 'ADELAIDE') & (df_Crash['Total Cas'] > 3)]
df_Adelaide.show()

+--------------------+----------+--------+--------+----------------+-----------+---------+----------+--------+--------+----+--------+--------+--------+----------+-------------+----------------+--------------+--------------------+------------+-------------+------------+--------+--------------+---------+------------+-------------+---------------+------------+--------------+----------+----------+--------------+
|           REPORT_ID|Stats Area|  Suburb|Postcode|        LGA Name|Total Units|Total Cas|Total Fats|Total SI|Total MI|Year|   Month|     Day|    Time|Area Speed|Position Type|Horizontal Align|Vertical Align|          Other Feat|Road Surface|Moisture Cond|Weather Cond|DayNight|    Crash Type|Unit Resp| Entity Code|CSEF Severity|  Traffic Ctrls|DUI Involved|Drugs Involved|  ACCLOC_X|  ACCLOC_Y|    UNIQUE_LOC|
+--------------------+----------+--------+--------+----------------+-----------+---------+----------+--------+--------+----+--------+--------+--------+----------+-------------+

**Display 10 crash events with highest casualties .**

In [81]:
# Total casualties based on crash type
Top10_Casualties = df_Crash.groupBy('Crash Type').agg(F.sum('Total Cas').alias('Total casualties')).sort('Total casualties', ascending=False)
# Displaying top 10 crash
Top10_Casualties.show(10)

+--------------------+----------------+
|          Crash Type|Total casualties|
+--------------------+----------------+
|            Rear End|          8605.0|
|         Right Angle|          6555.0|
|    Hit Fixed Object|          4502.0|
|          Right Turn|          3120.0|
|           Roll Over|          2279.0|
|          Side Swipe|          1992.0|
|      Hit Pedestrian|          1517.0|
|             Head On|          1224.0|
|  Hit Parked Vehicle|          1133.0|
|Left Road - Out o...|           244.0|
+--------------------+----------------+
only showing top 10 rows



**Find the total number of fatalities for each crash type.**

In [82]:
# Total fatalities based on crash type
df_CrashType = df_Crash.groupBy('Crash Type').agg(F.sum('Total Fats').alias('fatalities'))

# Displaying fatalities
df_CrashType.show()

+--------------------+----------+
|          Crash Type|fatalities|
+--------------------+----------+
|           Roll Over|      57.0|
|  Hit Object on Road|       2.0|
|      Hit Pedestrian|      70.0|
|    Hit Fixed Object|     152.0|
|               Other|       2.0|
|          Side Swipe|      20.0|
|             Head On|      86.0|
|  Hit Parked Vehicle|       9.0|
|          Right Turn|      18.0|
|            Rear End|      16.0|
|          Hit Animal|       4.0|
|Left Road - Out o...|       1.0|
|         Right Angle|      45.0|
+--------------------+----------+



**Find the total number of casualties for each suburb when the vehicle was driven by an
unlicensed driver. You are required to display the name of the suburb and the total
number of casualties.**

In [83]:
# Joining Unit and Crash data using Outer join
df_join = df_Crash.join(df_Units,df_Crash.REPORT_ID == df_Units.REPORT_ID, how = 'outer')
# Selecting required columns 'Suburb', 'Total Cas', 'Licence Type'
df_LicType = df_join.select(['Suburb', 'Total Cas', 'Licence Type'])
# Filtering data based on Licence Type' equals to 'Unlicenced'
df_UnLic = df_LicType[df_LicType['Licence Type'] == 'Unlicenced']
# Calculating total casualties for each suburb
df_CasualitiesSuburb = df_UnLic.groupby('Suburb').agg(F.sum('Total Cas').alias('casualties'))
df_CasualitiesSuburb.sort('casualties', ascending=False).show()

+---------------+----------+
|         Suburb|casualties|
+---------------+----------+
|       ADELAIDE|      19.0|
|      SALISBURY|      18.0|
|      DRY CREEK|      18.0|
| SALISBURY EAST|      16.0|
|       PROSPECT|      14.0|
| NORTH ADELAIDE|      13.0|
|        ENFIELD|      12.0|
|   ANDREWS FARM|      12.0|
|     INGLE FARM|      11.0|
|   BEDFORD PARK|      11.0|
|SALISBURY SOUTH|      11.0|
|SALISBURY DOWNS|      11.0|
|     MUNNO PARA|      10.0|
|SALISBURY PLAIN|      10.0|
|  MORPHETT VALE|      10.0|
|   MOUNT BARKER|      10.0|
|         BURTON|      10.0|
|   MAWSON LAKES|      10.0|
| ELIZABETH PARK|      10.0|
|    HOLDEN HILL|       9.0|
+---------------+----------+
only showing top 20 rows



**2.3 Severity Analysis**

**Find the total number of crash events for each severity level. Which severity level is the
most common?**

In [84]:
# Counting total number of crash events for each severity level
df_Severity = df_Crash.groupBy('CSEF Severity').agg(F.count('CSEF Severity').alias('Total Severity'))
df_Severity.show()

+-------------+--------------+
|CSEF Severity|Total Severity|
+-------------+--------------+
|     4: Fatal|           451|
|        2: MI|         21881|
|       1: PDO|         46696|
|        3: SI|          2978|
+-------------+--------------+



**Compute the total number of crash events for each severity level and the percentage
for the four different scenarios.**

In [85]:
# Counting total number of crash events for each severity level
df_SeverityCrash = df_Crash.groupBy('CSEF Severity').agg(F.count('CSEF Severity').alias('Total_Crash'))
# Finding total crash events
addition = df_SeverityCrash.groupBy().agg(F.sum("Total_Crash")).collect()[0][0]
# creating new column showing the percentage
df_NewCol = df_SeverityCrash.withColumn('Percentage',(df_SeverityCrash.Total_Crash/addition)*100)

df_NewCol.show()


+-------------+-----------+------------------+
|CSEF Severity|Total_Crash|        Percentage|
+-------------+-----------+------------------+
|     4: Fatal|        451|0.6263366941643752|
|        2: MI|      21881| 30.38774546565564|
|       1: PDO|      46696|  64.8501513762742|
|        3: SI|       2978| 4.135766463905786|
+-------------+-----------+------------------+



**When the driver is tested positive on drugs.**

In [86]:
# filtering data when driver tested positive for drugs
df_Drugs = df_Crash[df_Crash['Drugs Involved'] == 'Y']
# Counting total number of crash events
df_SeverityCrash = df_Drugs.groupBy('CSEF Severity').agg(F.count('Total Cas').alias('Positive on Drugs'))
# Finding total crash events
addition = df_SeverityCrash.groupBy().agg(F.sum("Positive on Drugs")).collect()[0][0]
# creating new column showing the percentage
df_NewCol = df_SeverityCrash.withColumn('Percentage',(df_SeverityCrash["Positive on Drugs"]/addition)*100)

df_NewCol.show()

+-------------+-----------------+------------------+
|CSEF Severity|Positive on Drugs|        Percentage|
+-------------+-----------------+------------------+
|     4: Fatal|               82| 6.539074960127592|
|        2: MI|              749|59.728867623604465|
|       1: PDO|              176|14.035087719298245|
|        3: SI|              247|19.696969696969695|
+-------------+-----------------+------------------+



**When the driver is tested positive for blood alcohol concentration.**

In [87]:
# filtering data when driver tested positive for alcohol
df_Alcohol = df_Crash[df_Crash['DUI Involved'] == 'Y']
# Counting total number of crash events
df_SeverityCrash = df_Alcohol.groupBy('CSEF Severity').agg(F.count('Total Cas').alias('Positive on Alcohol'))
# Finding total crash events
addition = df_SeverityCrash.groupBy().agg(F.sum("Positive on Alcohol")).collect()[0][0]
# creating new column showing the percentage
df_NewCol = df_SeverityCrash.withColumn('Percentage',(df_SeverityCrash["Positive on Alcohol"]/addition)*100)

df_NewCol.show()

+-------------+-------------------+------------------+
|CSEF Severity|Positive on Alcohol|        Percentage|
+-------------+-------------------+------------------+
|     4: Fatal|                 79|  3.51423487544484|
|        2: MI|                737|  32.7846975088968|
|       1: PDO|               1173|52.179715302491104|
|        3: SI|                259| 11.52135231316726|
+-------------+-------------------+------------------+



**When the driver is tested positive for both drugs and blood alcohol**

In [88]:
# filtering when driver is tested positive for both drugs and blood alcohol
df_Alcohol_Drugs = df_Crash[(df_Crash['DUI Involved'] == 'Y') & (df_Crash['Drugs Involved'] == 'Y')]
# Counting total number of crash events
df_SeverityCrash = df_Alcohol_Drugs.groupBy('CSEF Severity').agg(F.count('Total Cas').alias('Positive on Drugs/Alcohol'))
# Finding total crash events
addition = df_SeverityCrash.groupBy().agg(F.sum("Positive on Drugs/Alcohol")).collect()[0][0]
# creating new column showing the percentage
df_NewCol = df_SeverityCrash.withColumn('Percentage',(df_SeverityCrash["Positive on Drugs/Alcohol"]/addition)*100)

df_NewCol.show()

+-------------+-------------------------+------------------+
|CSEF Severity|Positive on Drugs/Alcohol|        Percentage|
+-------------+-------------------------+------------------+
|     4: Fatal|                       27|15.428571428571427|
|        2: MI|                       89|50.857142857142854|
|       1: PDO|                       24|13.714285714285715|
|        3: SI|                       35|              20.0|
+-------------+-------------------------+------------------+



**When the driver is tested negative for both (no alcohol and no drugs).**

In [89]:
# filtering when driver is tested negative for both drugs and blood alcohol
df_No_Alcohol_Drugs = df_Crash.filter((df_Crash['DUI Involved'].isNull()) & (df_Crash['Drugs Involved'].isNull()))
# Counting total number of crash events
df_SeverityCrash = df_No_Alcohol_Drugs.groupBy('CSEF Severity').agg(F.count('Total Cas').alias('Negative on Drugs/Alcohol'))
# Finding total crash events
addition = df_SeverityCrash.groupBy().agg(F.sum("Negative on Drugs/Alcohol")).collect()[0][0]
# creating new column showing the percentage
df_NewCol = df_SeverityCrash.withColumn('Percentage',(df_SeverityCrash["Negative on Drugs/Alcohol"]/addition)*100)

df_NewCol.show()

+-------------+-------------------------+------------------+
|CSEF Severity|Negative on Drugs/Alcohol|        Percentage|
+-------------+-------------------------+------------------+
|     4: Fatal|                      317| 0.461567582521586|
|        2: MI|                    20484|29.825710916000524|
|       1: PDO|                    45371| 66.06240626683557|
|        3: SI|                     2507|3.6503152346423215|
+-------------+-------------------------+------------------+



**2.4 RDDs vs DataFrame vs Spark SQL**

**Find the Date and Time of Crash, Number of Casualties in each unit and the Gender,
Age, License Type of the unit driver for the suburb "Adelaide".**

In [90]:
%%time
# Creating key value pair based  on REPORT_ID
def keyValue(li):
    return (li[0], li[1:])

Unit_keyValue = Unit_RDD1.map(keyValue)
Crash_keyValue = Crash_RDD1.map(keyValue)

# Joining Unit and Crash RDD based on Key REPORT_ID
joined_RDD = Unit_keyValue.join(Crash_keyValue)
# Filtering required columns for analysis
RDD = joined_RDD.map(lambda x: (x[1][0][0],x[1][0][1],x[1][0][3],x[1][0][6],x[1][0][7],x[1][0][10],x[1][1][1],x[1][1][9],
                                       x[1][1][10],x[1][1][11],x[1][1][12]))

# Selecting columns Date, Time, Number of Casualties, Gender, Age, License Type for the suburb "Adelaide".
NoOfCaus = RDD.filter(lambda x: x[6] == 'ADELAIDE').map(lambda x: (x[7]+"-"+x[8]+"-"+x[9],x[10],x[0],x[1],x[3],x[4],x[5]))
NoOfCaus.take(10)

CPU times: user 43 ms, sys: 4.29 ms, total: 47.3 ms
Wall time: 4.63 s


[('2016-November-Wednesday', '04:26 pm', '01', '0', 'Male', '017', 'Unknown'),
 ('2016-November-Wednesday', '04:26 pm', '02', '0', 'Male', '025', 'Unknown'),
 ('2016-December-Friday', '11:30 am', '01', '0', 'Male', '080', 'Full'),
 ('2016-December-Friday', '11:30 am', '02', '0', 'Male', '048', 'Full'),
 ('2016-December-Saturday', '07:40 am', '01', '0', 'Male', '032', 'Full'),
 ('2016-December-Saturday',
  '07:40 am',
  '02',
  '0',
  'Unknown',
  'XXX',
  'Unknown'),
 ('2016-December-Friday', '05:30 pm', '01', '0', 'Female', '058', 'Full'),
 ('2016-December-Friday', '05:30 pm', '02', '0', 'Male', '041', 'Full'),
 ('2016-December-Wednesday', '04:20 pm', '01', '0', 'Female', '045', 'Full'),
 ('2016-December-Wednesday', '04:20 pm', '02', '0', 'Male', '027', 'Full')]

In [91]:
%%time
# filtering data based on Suburb ADELAIDE
df_Adelaide = df_join.filter(df_join['Suburb'] == 'ADELAIDE')

x = "-" # character to join Year Month Day
# Selecting columns Date, Time, Number of Casualties, Gender, Age, License Type for the suburb "Adelaide"
df_Adelaide = df_Adelaide.select(F.concat('Year',F.lit(x),'Month',F.lit(x),'Day').alias('Date'),'Time','Unit No','No Of Cas','Sex','Age','Licence Type')

df_Adelaide.show(10)

+--------------------+--------+-------+---------+-------+----+------------+
|                Date|    Time|Unit No|No Of Cas|    Sex| Age|Licence Type|
+--------------------+--------+-------+---------+-------+----+------------+
|2016-November-Wed...|01:45 pm|     02|        1|   Male| 072|        null|
|2016-November-Wed...|01:45 pm|     01|        0|   Male| 056|        Full|
|2016-November-Tue...|03:40 pm|     02|        1| Female| 027|        null|
|2016-November-Tue...|03:40 pm|     01|        0|   Male| 056|        null|
|2016-November-Tue...|05:00 pm|     02|        0|Unknown| XXX|     Unknown|
|2016-November-Tue...|05:00 pm|     01|        0| Female| 032|        Full|
|2016-November-Tue...|05:40 pm|     02|        0|   Male| 020|     Unknown|
|2016-November-Tue...|05:40 pm|     01|        0|   Male| 022|     Unknown|
|2016-November-Monday|11:26 pm|     03|        0|   null|null|        null|
|2016-November-Monday|11:26 pm|     02|        0|   Male| 042|        Full|
+-----------

In [92]:
%%time
# Creating Temporary view for both Unit and Crash Data
df_Units.createOrReplaceTempView("UnitData")
df_Crash.createOrReplaceTempView("CrashData")

# Selecting columns Date, Time, Number of Casualties, Gender, Age, License Type for the suburb "Adelaide"
SqlAdelaide = spark.sql('''
SELECT CONCAT(Year, '-', Month, '-', Day) as Date, Time,'Unit No','No Of Cas', Sex, Age,'Licence Type'
FROM UnitData u, CrashData c 
WHERE u.REPORT_ID = c.REPORT_ID and Suburb == 'ADELAIDE'
''')

SqlAdelaide.show(10)

+--------------------+--------+-------+---------+-------+---+------------+
|                Date|    Time|Unit No|No Of Cas|    Sex|Age|Licence Type|
+--------------------+--------+-------+---------+-------+---+------------+
|2016-November-Wed...|01:45 pm|Unit No|No Of Cas|   Male|056|Licence Type|
|2016-November-Wed...|01:45 pm|Unit No|No Of Cas|   Male|072|Licence Type|
|2016-November-Tue...|03:40 pm|Unit No|No Of Cas|   Male|056|Licence Type|
|2016-November-Tue...|03:40 pm|Unit No|No Of Cas| Female|027|Licence Type|
|2016-November-Tue...|05:00 pm|Unit No|No Of Cas| Female|032|Licence Type|
|2016-November-Tue...|05:00 pm|Unit No|No Of Cas|Unknown|XXX|Licence Type|
|2016-November-Tue...|05:40 pm|Unit No|No Of Cas|   Male|022|Licence Type|
|2016-November-Tue...|05:40 pm|Unit No|No Of Cas|   Male|020|Licence Type|
|2016-November-Monday|11:26 pm|Unit No|No Of Cas|Unknown|XXX|Licence Type|
|2016-November-Monday|11:26 pm|Unit No|No Of Cas|   Male|042|Licence Type|
+--------------------+---

**Find the total number of casualties for each suburb when the vehicle was driven by an
unlicensed driver. You are required to display the name of the suburb and the total
number of casualties.**

In [101]:
%%time
# Joining Unit and Crash data using Outer join
df_join = df_Crash.join(df_Units,df_Crash.REPORT_ID == df_Units.REPORT_ID, how = 'outer')
# Selecting required columns 'Suburb', 'Total Cas', 'Licence Type'
df_LicType = df_join.select(['Suburb', 'Total Cas', 'Licence Type'])
# Filtering data based on Licence Type' equals to 'Unlicenced'
df_UnLic = df_LicType[df_LicType['Licence Type'] == 'Unlicenced']
# Calculating total casualties for each suburb
df_CasualitiesSuburb = df_UnLic.groupby('Suburb').agg(F.sum('Total Cas').alias('Casualties'))
# displaying top 10 rows
df_CasualitiesSuburb.sort('Casualties', ascending=False).show(10)

+---------------+----------+
|         Suburb|Casualties|
+---------------+----------+
|       ADELAIDE|      19.0|
|      DRY CREEK|      18.0|
|      SALISBURY|      18.0|
| SALISBURY EAST|      16.0|
|       PROSPECT|      14.0|
| NORTH ADELAIDE|      13.0|
|        ENFIELD|      12.0|
|   ANDREWS FARM|      12.0|
|SALISBURY SOUTH|      11.0|
|SALISBURY DOWNS|      11.0|
+---------------+----------+
only showing top 10 rows

CPU times: user 10.1 ms, sys: 156 µs, total: 10.3 ms
Wall time: 1.74 s


In [95]:
%%time
# Creating key value pair based  on REPORT_ID
def keyValue(li):
    return (li[0], li[1:])

Unit_keyValue = Unit_RDD1.map(keyValue)
Crash_keyValue = Crash_RDD1.map(keyValue)

# Joining Unit and Crash RDD based on Key REPORT_ID
joined_RDD = Unit_keyValue.join(Crash_keyValue)

# Selecting required columns 'Suburb', 'Total Cas', 'Licence Type'
RDD1 = joined_RDD.map(lambda x: (x[1][0][10],x[1][1][1],x[1][1][4],x[1][1][5]))
# Filtering data based on Licence Type' equals to 'Unlicenced'
RDD1 = RDD1.filter(lambda x: x[0]=='Unlicenced')
# Converting data type for total cas to integer
RDD1 = RDD1.map(lambda x: (x[1],int(x[3])))
# Group by suburb and taking sum of Total Cas
RDD1 = RDD1.groupByKey().mapValues(sum)
# Sorting by key that is Suburb, top 10 rows
RDD1.takeOrdered(10, key = lambda x: -x[1])

CPU times: user 64.5 ms, sys: 15.1 ms, total: 79.6 ms
Wall time: 5.84 s


[('ADELAIDE', 19),
 ('KENT TOWN', 19),
 ('DRY CREEK', 18),
 ('SALISBURY', 18),
 ('PAYNEHAM', 17),
 ('NORWOOD', 17),
 ('SALISBURY EAST', 16),
 ('MARDEN', 16),
 ('STEPNEY', 15),
 ('MAYLANDS', 14)]

In [102]:
%%time
# Selecting columns 'Suburb', 'Total Cas', 'Licence Type' and group by Licence Type
SqlCasualities = spark.sql('''
SELECT Suburb, sum(`Total Cas`) as `Casualties` 
FROM UnitData u, CrashData c 
where u.REPORT_ID = c.REPORT_ID and `Licence Type` == "Unlicenced" 
GROUP BY Suburb ORDER BY `Casualties` DESC 
''')
# displaying top 10 rows
SqlCasualities.show(10)

+---------------+----------+
|         Suburb|Casualties|
+---------------+----------+
|       ADELAIDE|      19.0|
|      SALISBURY|      18.0|
|      DRY CREEK|      18.0|
| SALISBURY EAST|      16.0|
|       PROSPECT|      14.0|
| NORTH ADELAIDE|      13.0|
|   ANDREWS FARM|      12.0|
|        ENFIELD|      12.0|
|SALISBURY DOWNS|      11.0|
|     INGLE FARM|      11.0|
+---------------+----------+
only showing top 10 rows

CPU times: user 2.92 ms, sys: 249 µs, total: 3.17 ms
Wall time: 1.76 s
